# Load Model and Data

# Very Important: Check this out: is your dataset shuffled properly !!!!

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch 
from dreamerv4.datasets import ShardedHDF5Dataset
from dreamerv4.models.utils import load_tokenizer
from dreamerv4.models.utils import load_denoiser
# DATA_PATH = "/home/mim-server/datasets/soar_data_sharded"
# DATA_PATH = "/home/mim-server/datasets/pushcube-play-hdf5"
DATA_PATH = "/home/mim-server/datasets/pushT/sharded"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resolution = (256, 256)

In [3]:
from hydra import initialize, compose
from omegaconf import OmegaConf
with initialize(version_base=None, config_path="../scripts/config"):
    cfg = compose(config_name="dynamics/pushT.yaml")

/home/mim-server/miniconda3/envs/dreamerv4/lib/python3.11/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'dynamics/pushT.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [4]:
dynamics_ckpt = "/home/mim-server/projects/rooholla/dreamer-v4-draft/checkpoints/dynamics_ckpts/dynamics-pushT-110M.pt"
cfg.dynamics_ckpt = dynamics_ckpt

In [5]:
tokenizer_ckpt="/home/mim-server/projects/rooholla/dreamer-v4-draft/checkpoints/tokenizer_ckpts/pushT.pt"
cfg.tokenizer_ckpt=tokenizer_ckpt

In [6]:
denoiser = load_denoiser(cfg, device, max_num_forward_steps=300)
tokenizer = load_tokenizer(cfg, device, max_num_forward_steps=300)
tokenizer = tokenizer.eval().cuda()
denoiser = denoiser.eval().cuda()

In [7]:
import mediapy
from torch.nn.functional import interpolate

dataset = ShardedHDF5Dataset(
        data_dir=DATA_PATH,
        window_size=64,
        stride=1,
        split='train',
        train_fraction=0.9,
        split_seed=123,
    )

# Arbitrary index into that episode
batch = dataset[300]
imgs = batch["image"]  # (T, C, H, W)
actions = batch["action"][:,:cfg.denoiser.n_actions]  # (1, T, N_lat, D_lat)
# actions=torch.zeros_like(actions)
imgs = interpolate(imgs, resolution).to(device=device)[None] # resize to tokenizer resolution

with torch.no_grad():
    with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
        latents = tokenizer.encode(imgs)
        imgs_recon = tokenizer.decode(latents)

Train split: 64694 windows from 11 episodes


In [11]:
from mediapy import show_video
def plotVideo(video):
    imgs = video.cpu().permute(0,2,3,1).to(torch.float32).numpy()*255
    imgs = imgs.astype('uint8')
    mediapy.show_video(imgs, fps=10)

plotVideo(imgs_recon[0])
plotVideo(imgs[0])

This browser does not support the video tag.


This browser does not support the video tag.


In [67]:
from dreamerv4.sampling import forward_dynamics_no_cache, AutoRegressiveForwardDynamics

In [69]:
num_pred_steps=24
latents_ctx = latents[:, :4, ...]
cond_actions = actions[:latents_ctx.shape[1]+num_pred_steps][None]
with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
    z = forward_dynamics_no_cache(denoiser, 
                                latents_ctx,
                                cond_actions,
                                num_pred_steps=num_pred_steps,
                                num_diffusion_steps=4)

    with torch.no_grad():
        img_pred = tokenizer.decode(z)


plotVideo(img_pred[0].to(torch.float32))

This browser does not support the video tag.


In [70]:
world = AutoRegressiveForwardDynamics(denoiser, tokenizer, device=device, dtype=torch.bfloat16)
with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
    world.reset(imgs[:,:8], actions[:8][None])

In [71]:
latents = []
imgs_pred = []
for i in range(32):
    action_t = actions[world.current_frame_index][None]
    # action_t[:, 0] = 1
    with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
        img = world.step(action_t)
    latents.append(world.current_z.clone())
    imgs_pred.append(img)
z = torch.cat(latents, dim=1)
imgs_pred = torch.stack(imgs_pred, dim=1)

In [72]:
with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.bfloat16):
    # z_decoded = tokenizer.decoder(world.latents_cond)
    z_decoded = tokenizer.decoder(z)
    imgs_recon = tokenizer.image_head(z_decoded)
    imgs_recon = (imgs_recon + 1.0) / 2.0  # [-1,1] → [0,1]
    imgs_recon = torch.clamp(imgs_recon, 0.0, 1.0)
plotVideo(imgs_recon[0])

This browser does not support the video tag.


In [48]:
plotVideo(imgs_pred[0])

This browser does not support the video tag.


In [12]:
import mediapy
from torch.nn.functional import interpolate

dataset = ShardedHDF5Dataset(
        data_dir=DATA_PATH,
        window_size=64,
        stride=1,
        split='train',
        train_fraction=0.9,
        split_seed=123,
    )

Train split: 64694 windows from 11 episodes


In [13]:
imgs1 = dataset[0]['image']
imgs2 = dataset[10]['image']

In [14]:
plotVideo(imgs1)

This browser does not support the video tag.


In [15]:
plotVideo(imgs2)

This browser does not support the video tag.


In [24]:
import random
random.shuffle(dataset.windows)